<a href="https://colab.research.google.com/github/iuliivasilev/SpamFilter/blob/main/EDA/SpamFilterParser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Примеры внешних фильтров в CommuniGate

1. [Исходная документация (UI)](https://www.communigate.ru/CommuniGatePro/russian/VirusScan.html#Launch)
2. [Пример реализации Spam Filter для CommuniGate Pro (Perl, но можно посмотреть как встраивать)](https://github.com/TFF-Enterprises/CGPSA)
3. [Внутренняя кухня CommuniGate](https://habr.com/ru/companies/communigatepro/articles/197720/)
4. [Поддерживаемый пример детектора спама (C/C++)](https://github.com/freiz/terminator)

In [ ]:
import pandas as pd
import numpy as np
import os
import re

### Набор данных

1. [Источник](https://www.kaggle.com/datasets/maharshipandya/email-spam-dataset-extended/code)
2. [Пример обработки](https://www.kaggle.com/code/maharshipandya/email-spam-classification-98)

In [ ]:
!pip install opendatasets
import opendatasets as od

od.download("https://www.kaggle.com/datasets/maharshipandya/email-spam-dataset-extended")

# user: iuliivasilievmsurus
# key: d2b6c216f2b747e4ebadb039821aa9ae

Skipping, found downloaded files in "./email-spam-dataset-extended" (use force=True to force download)


In [ ]:
f = open('/content/email-spam-dataset-extended/ham_zipped/main_ham/00001.1a31cc283af0060967a233d26548a6ce', 'rb+')
s = f.read()

In [ ]:
### Документация парсера: https://docs.python.org/3/library/email.parser.html
import email
msg = email.message_from_bytes(s)
msg.__dict__

{'policy': Compat32(),
 '_headers': [('Return-Path', '<exmh-workers-admin@spamassassin.taint.org>'),
  ('Delivered-To', 'yyyy@localhost.netnoteinc.com'),
  ('Received',
   'from localhost (localhost [127.0.0.1])\n\tby phobos.labs.netnoteinc.com (Postfix) with ESMTP id 7106643C34\n\tfor <jm@localhost>; Wed, 21 Aug 2002 08:33:03 -0400 (EDT)'),
  ('Received',
   'from phobos [127.0.0.1]\n\tby localhost with IMAP (fetchmail-5.9.0)\n\tfor jm@localhost (single-drop); Wed, 21 Aug 2002 13:33:03 +0100 (IST)'),
  ('Received',
   'from listman.spamassassin.taint.org (listman.spamassassin.taint.org [66.187.233.211]) by\n    dogma.slashnull.org (8.11.6/8.11.6) with ESMTP id g7LCXvZ24654 for\n    <jm-exmh@jmason.org>; Wed, 21 Aug 2002 13:33:57 +0100'),
  ('Received',
   'from listman.spamassassin.taint.org (localhost.localdomain [127.0.0.1]) by\n    listman.redhat.com (Postfix) with ESMTP id F12A13EA25; Wed, 21 Aug 2002\n    08:34:00 -0400 (EDT)'),
  ('Delivered-To', 'exmh-workers@listman.spamassass

In [ ]:
def set_prefix(l, prefix):
    return [f"{prefix}_{e}" for e in l]

def get_content(msg):
    if msg.is_multipart():
        for part in msg.walk():
            ctype = part.get_content_type()
            cdispo = str(part.get('Content-Disposition'))

            # skip text/plain type
            if ctype == 'text/plain' and 'attachment' not in cdispo:
                body = part.get_payload(decode=True)  # decode
                break
    else:
        body = msg.get_payload(decode=True)
    content = body.decode().lower()
    content = content.replace("=\n", "")
    content = re.sub(r"\s+", " ", content)
    return {"Content": content, "Subject": msg["Subject"].lower()}

def get_context(msg):
    return {}

def collect_by_folder(path, type_folder=None):
    l = []
    for file in os.listdir(path):
        try:
            with open(os.path.join(path, file), 'rb+') as f:
                s = f.read()
            msg = email.message_from_bytes(s)
            d_msg = {"id": file}
            d_msg.update(get_content(msg))
            d_msg.update(get_context(msg))
            if not(type_folder is None):
                d_msg["type"] = type_folder
            l.append(d_msg)
        except Exception as e:
            print("Error in file:", file)
    return l

In [ ]:
l_msg = collect_by_folder("/content/email-spam-dataset-extended/ham_zipped/main_ham", type_folder="ham")
l_msg += collect_by_folder("/content/email-spam-dataset-extended/spam_zipped/main_spam", type_folder="spam")

Error in file: 00664.28f4cb9fad800d0c7175d3a67e6c6458
Error in file: 1303.b8fb03a2de8615f9287086ccc71a4282
Error in file: 0891.44f5cad6f0dfa66277e1a0aec55e7bf6
Error in file: 00180.1d7edb742cfbd5d48881c99a8e03cccd
Error in file: 00499.b66750ed646219a0524a648a061dfa67
Error in file: 0773.e1edafa86a1670e9515d4d9c1bb288c9
Error in file: 02026.e6e094c6110cbff0c3a55e0fc5c9273a
Error in file: 0287.18f9a821d67a503c0d54a2356a0f8a4a
Error in file: 0196.9dc01775acba34e580ddf9a56ea6891e
Error in file: 00083.5c1fe69b6ebb360baac59ddca2b9bda0
Error in file: 00169.701617763832617a7d0d1dfbc08b8a0d
Error in file: 00500.2c54eea1fb7f8bad057871a317212ad6
Error in file: 00581.9922b0f34aa8f51fb4adf4bf4ecf1464
Error in file: 02340.de68a83f900418f6cde87bdf92801f73
Error in file: 01128.efb36914ecb55d78a894591eff0843c5
Error in file: 00378.6634f5d52bac8536d979753841ade35a
Error in file: 00548.9df9bd35a18874dcf39ec227a063b847
Error in file: 0457.dc1691cbb334cc33a1f1eb3060b8e02e
Error in file: 01156.64602d74854cb

In [ ]:
df = pd.DataFrame(l_msg)

In [ ]:
content = df["Content"]
subject = df["Subject"]

## Обработка

In [ ]:
### Использование кастомного словаря

# import nltk
# nltk.download('punkt')
# nltk.download('stopwords')

# from nltk.tokenize import sent_tokenize, word_tokenize
# from nltk.corpus import stopwords

# data = df["Content"]
# stopWords = set(stopwords.words('english') + stopwords.words('russian'))
# words = word_tokenize(data)
# wordsFiltered = [w for w in words if w not in stopWords]

# filtered_body = " ".join(wordsFiltered)

#### CountVectorizer

1. [Документация](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html#sklearn.feature_extraction.text.CountVectorizer.fit)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(stop_words="english", ngram_range=(2, 2), max_features=5000)
X_ngramm = vectorizer.fit_transform(content)
features = set_prefix(vectorizer.get_feature_names_out(), "ngramm")
df_ngramm1 = pd.DataFrame(X_ngramm.toarray(), columns=features)

X_ngramm = vectorizer.transform(subject)
features_subj = set_prefix(vectorizer.get_feature_names_out(), "subj_ngramm")
df_ngramm2 = pd.DataFrame(X_ngramm.toarray(), columns=features_subj)

df_ngramm = pd.concat([df_ngramm1, df_ngramm2], axis=1)
df_ngramm

,ngramm_00 00,ngramm_00 000,ngramm_00 01,ngramm_00 12,ngramm_00 23,ngramm_00 asp,ngramm_00 br,ngramm_00 font,ngramm_00 html,ngramm_00 img,...,subj_ngramm_zdnet spamassassin,subj_ngramm_zdnet td,subj_ngramm_zdnetshopper cnet,subj_ngramm_zero length,subj_ngramm_zero sum,subj_ngramm_zip code,subj_ngramm_zoom br,subj_ngramm_zoom font,subj_ngramm_zzzz example,subj_ngramm_zzzz spamassassin
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8382,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8383,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8384,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8385,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### TfidfVectorizer

1. [Документация](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(stop_words="english", max_features=5000)
X_tfidf = tfidf_vectorizer.fit_transform(content)
features = set_prefix(tfidf_vectorizer.get_feature_names_out(), "tfidf")
df_tfidf1 = pd.DataFrame(X_tfidf.toarray(), columns=features)

X_tfidf = tfidf_vectorizer.transform(subject)
features_subj = set_prefix(tfidf_vectorizer.get_feature_names_out(), "subj_tfidf")
df_tfidf2 = pd.DataFrame(X_tfidf.toarray(), columns=features_subj)

df_tfidf = pd.concat([df_tfidf1, df_tfidf2], axis=1)
df_tfidf

,tfidf_00,tfidf_000,tfidf_0000,tfidf_000000,tfidf_000033,tfidf_000066,tfidf_000080,tfidf_000099,tfidf_0000a0,tfidf_0000cc,...,subj_tfidf_zawodny,subj_tfidf_zdnet,subj_tfidf_zero,subj_tfidf_ziggy,subj_tfidf_zimbabwe,subj_tfidf_zip,subj_tfidf_zone,subj_tfidf_zoom,subj_tfidf_zope,subj_tfidf_zzzz
0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000000,0.0,0.087033,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8382,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8383,0.000000,0.0,0.000000,0.089411,0.0,0.0,0.030791,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8384,0.003152,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8385,0.000000,0.0,0.000000,0.103694,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#### Word2vec

1. [Теория + базовые возможности](https://builtin.com/machine-learning/nlp-word2vec-python)
2. [Документация](https://radimrehurek.com/gensim/models/word2vec.html)
3. [Базовый MeanEmbeddingVectorizer](https://nadbordrozd.github.io/blog/2016/05/20/text-classification-with-word2vec/)

In [ ]:
from gensim.models import Word2Vec

class MeanEmbeddingVectorizer(object):
    def __init__(self, **kwargs):
        self.kwargs = kwargs

    def prepare_text(self, X):
        return list(map(lambda x: x.split(), X))

    def fit(self, X):
        X_by_word = self.prepare_text(X)
        model = Word2Vec(sentences=X_by_word, **self.kwargs)
        self.word2vec_ki = model.wv.key_to_index
        self.word2vec_v = model.wv.vectors
        self.word2vec_s = model.wv.index_to_key

        self.dim = model.vector_size
        return self

    def transform(self, X):
        X_by_word = self.prepare_text(X)
        return np.array([
            np.mean([self.word2vec_v[self.word2vec_ki[w]] for w in words if w in self.word2vec_s]
                    or [np.zeros(self.dim)], axis=0)
            for words in X_by_word
        ])

    def get_feature_names_out(self):
        return list(map(str, np.arange(self.dim)))

In [ ]:
w2v_vectorizer = MeanEmbeddingVectorizer(vector_size=100, window=5, min_count=1, max_final_vocab=1000)
w2v_vectorizer.fit(content)

X_w2v = w2v_vectorizer.transform(content)
features = set_prefix(w2v_vectorizer.get_feature_names_out(), "w2v")
df_w2v1 = pd.DataFrame(X_w2v, columns=features)

X_w2v = w2v_vectorizer.transform(subject)
features_subj = set_prefix(w2v_vectorizer.get_feature_names_out(), "subj_w2v")
df_w2v2 = pd.DataFrame(X_w2v, columns=features_subj)

df_w2v = pd.concat([df_w2v1, df_w2v2], axis=1)
df_w2v

,w2v_0,w2v_1,w2v_2,w2v_3,w2v_4,w2v_5,w2v_6,w2v_7,w2v_8,w2v_9,...,subj_w2v_90,subj_w2v_91,subj_w2v_92,subj_w2v_93,subj_w2v_94,subj_w2v_95,subj_w2v_96,subj_w2v_97,subj_w2v_98,subj_w2v_99
0,0.148887,0.397153,0.425654,0.262848,0.413787,-0.249922,-0.079366,0.406009,-0.236737,0.056725,...,-0.583367,-0.396436,0.349486,-0.947634,0.666325,-0.479923,1.086828,-0.538667,0.502130,-0.643400
1,0.111065,0.372262,0.403161,0.090790,0.475632,0.102723,0.210201,0.138674,-0.262204,-0.220159,...,0.117044,0.009654,0.216197,-0.086107,0.462271,0.508599,0.873253,-0.557903,0.431626,0.447825
2,0.031703,0.329090,0.621229,-0.026359,0.506702,-0.206374,0.069078,0.142992,-0.352894,0.216167,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.021016,0.508100,0.450320,0.172715,0.431491,0.021096,0.254222,0.173868,-0.420919,0.082307,...,0.479818,-0.110164,0.655332,0.536347,0.049571,0.366604,-0.048589,0.580585,0.444693,0.891965
4,0.267555,0.277769,-0.246213,0.152574,-0.358634,-0.784510,-0.288577,0.678145,-0.791729,0.820232,...,-0.416869,-0.251121,-0.888340,-0.474692,-0.203191,-0.852486,0.008018,-0.421527,0.228255,-0.206238
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8382,0.148764,-0.005687,0.179707,0.201552,0.796642,-0.124200,0.161964,0.352255,-0.377505,0.123007,...,0.200691,0.180255,-1.085723,0.624863,-0.406194,-0.986258,-0.474732,-0.492851,-0.480147,0.172107
8383,-0.039107,-0.523866,-0.385058,0.181705,0.261816,-0.605469,0.095054,0.463872,0.164578,-0.126135,...,-0.396678,-1.031199,-0.812809,-0.232370,0.061501,-1.078642,0.553175,-0.787260,0.085643,-0.611634
8384,-0.132537,0.324673,-0.087395,0.743984,-0.047120,-0.565276,-0.695792,0.305624,-0.139365,0.113976,...,0.758200,-1.324002,0.417787,-0.906096,1.167264,-1.072425,1.135059,-0.209545,0.367039,-0.204262
8385,0.272295,-0.326448,-0.246503,0.431680,0.038799,-0.571532,0.509921,0.601057,-0.413321,0.340169,...,0.225687,0.448669,0.121024,-0.310529,0.168675,-0.132850,-0.131762,-0.486448,-0.120408,0.335867


In [ ]:
# X = pd.concat([df_ngramm, df_tfidf, df_w2v], axis=1)
# y = df["type"] == "spam"

X = pd.concat([df_ngramm, df_tfidf, df_w2v], axis=1)
y = df["type"] == "spam"

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=42)

print("Training set size: ", X_train.shape, y_train.shape)
print("Testing set size: ", X_test.shape, y_test.shape)

Training set size:  (6709, 20200) (6709,)
Testing set size:  (1678, 20200) (1678,)


In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(random_state=42)
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

print("Accuracy score: ", accuracy_score(y_test, y_pred))
print("Classification report:\n", classification_report(y_test, y_pred, digits=6))

Accuracy score:  0.9922526817640047
Classification report:
               precision    recall  f1-score   support

       False   0.992366  0.997698  0.995025      1303
        True   0.991848  0.973333  0.982503       375

    accuracy                       0.992253      1678
   macro avg   0.992107  0.985515  0.988764      1678
weighted avg   0.992251  0.992253  0.992227      1678



In [ ]:
0.99046 # ngrams
0.98986 # + tfdif
0.99225 # + word2vec